Correction du TD 1
https://colab.research.google.com/drive/12HX8Hytszb81gagT6cws6S9XQgY1IqQO

# Iterables

- Definition : quelque chose sur lequel on peut itérer
- Nature : abstraite, un iterable peut être une liste, un générateur etc...
- Utilisation "automatique" : `for x in mon_iterable`

Que se passe t-il derrière `for x in mon_iterable:` ?
- Python apelle iter(mon_iterable)
    - necessite la définition `mon_iterable.__iter__`
- Il obtient un iterator
    - Qu'il utilise avec next(iterator)
    - Necessite la définition `iterator.__next__`

## Demonstration

In [ ]:
from collections.abc import Iterable
an_iterable = range(1, 10**100)

In [ ]:
print(isinstance(an_iterable, Iterable))
print(hasattr(an_iterable, "__iter__"))

In [ ]:
an_iterator = iter(an_iterable)
print(type(an_iterator))
print(hasattr(an_iterator, "__next__"))

In [ ]:
print(next(an_iterator))
print(next(an_iterator)) 

# Comment faire ses propres iterables

- Solution coûteuse : Implémenter `__iter__` retournant un objet implémentant `__next__` (et qui soulève l'exception `StopIteration` lorsqu'il n'y a plus d'elements) 
- Solution simple et pratique : **Composer son iterateur à partir d'un (ou plusieurs) autres**. Quatre possibilités : 
    - Utiliser iter sur un autre iterateur
    - Utiliser yield
    - Utiliser un "Generator Expression"
    - Utiliser itertools pour une solution éléguante à un cas plus complexe

In [ ]:
class StudentsClass:
    def __init__(self, promotion, students_list=[]):
        self.promotion = promotion
        self.students_list = students_list
    
    def __iter__(self):
        return iter(self.students_list)
        # comme self.students_list est un iterable, on peut l'utiliser directement

In [ ]:
myclass = StudentsClass("epitaMTI", ["Joe", "Tom", "Lila"])
for s in myclass:
    print(s) 

**Question** : Comment faire en sorte d'éviter certains étudiants (manquants ?)

In [ ]:
class StudentsClass:
    def __init__(self, promotion, students_list=[]):
        self.promotion = promotion
        self.students_list = students_list
        self.missing_students = []
        
    def add_missing_student(self, stud):
        self.missing_students.append(stud)

myclass = StudentsClass("epitaMTI", ["Joe", "Tom", "Lila"])
myclass.add_missing_student("Tom")

# COMMENT implémenter__iter__ de telle sorte que Tom soit évité lorsque j'itère sur myclass ?

In [ ]:
class StudentsClass:
    def __init__(self, promotion, students_list=[]):
        self.promotion = promotion
        self.students_list = students_list
        self.missing_students = []
        
    def add_missing_student(self, stud):
        self.missing_students.append(stud)
    
    def __iter__(self):
        for s in self.students_list:
            if s not in self.missing_students:
                yield s

myclass = StudentsClass("epitaMTI", ["Joe", "Tom", "Lila"])
myclass.add_missing_student("Tom")
for s in myclass:
    print(s)

Le mot clef `yield` permet de produire une fonction retournant un iterable (sans se soucier de produire plus manuellement un iterator)

In [ ]:
import random
print(random.random())

In [ ]:
def generate_random_numbers(nmax):
    for n in range(nmax):
        yield random.random()

In [ ]:
type(generate_random_numbers(10))

`generate_random_numbers` n'est pas une fonction comme les autres. La présence du mot clef `yield` implique qu'elle va produire un generateur.

Solution alternative : produire un iterable à partir de range.

Fonctionne comme les list comprehension, mais la présence de parenthèses permet de construire un iterable.

In [ ]:
def generate_random_numbers(nmax):
    return (random.random() for _ in range(nmax))
print(type(generate_random_numbers(10)))

In [ ]:
for f in generate_random_numbers(10):
    print(f)

Comparaison avec les listes:

- generate_random_numbers(10) se comporte comme une liste dans le cadre d'une boucle for
- mais les 10 éléments n'auront jamais été chargé en mémoire en même temps (un par un)

Pour aller plus loin. Le module [itertools](https://docs.python.org/3.7/library/itertools.html) offre un large pannel d'operations sur les iterables (cycler, combiner, permuter, filtrer, produit cartésien etc...)

# Les dangers des valeurs par défaut

Rappel

In [ ]:
def add(a, b=0):
    return a + b

print(add(5))
print(add(5, 5))

In [ ]:
def add(a, b=1+1):
    return a + b

In [ ]:
add.__defaults__[0]

La valeur par défaut est évaluée et stockée dans `add.__defaults__` dès l'interprétation de la fonction (pas au moment de l'execution)

Cela peut avoir des comportements suprenants

In [ ]:
from datetime import datetime
def some_operation_on_dates(a_date, now=datetime.now()):
    pass

In [ ]:
some_operation_on_dates.__defaults__[0]

Donc, même si la fonction est apellée tardivement, la valeur par défaut sera toujours `datetime.datetime(2019, 3, 18, 10, 26, 26, 257275)` 

**Solution ?**

In [ ]:
def some_operation_on_dates(a_date, now=None):
    if now is None: 
        now = datetime.now()
    pass

A retenir : La valeur par défaut est définie lors de l'interpretation de la fonction et est stockée dans `__defaults__`

## Retour sur le problème du bus

In [ ]:
class Bus:
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.insert(0, name)
        
    def drop(self, name):
       # if s in self.passengers: self.passengers.remove(s)
        self.passengers.remove(name)

La liste vide est passée comme valeur par défaut si passengers n'est pas fournie.

Cette classe a un problème absolument fondamental, lequel ?

In [ ]:
# Que va me donner le résultat de cette comparaison ?

bus1 = Bus()
print(Bus.__init__.__defaults__[0] is bus1.passengers)

In [ ]:
bus1.pick("Carrie")
bus2 = Bus()
print(bus2.passengers)

**Solution ?**

In [ ]:
class Bus:
    def __init__(self, passengers=[]):
        self.passengers = list(passengers)
        # self.passengers n'est plus un alias pour la valeur par défaut de la fonction
        # self.passengers est maintenant une copie de cette valeur par défaut 
        
    def pick(self, name):
        self.passengers.insert(0, name)
        
    def drop(self, name):
       # if s in self.passengers: self.passengers.remove(s)
        self.passengers.remove(name)

Leçon : être **très vigilant** lorsque des mutables sont utilisés comme valuer par défaut.